## Import CSV

In [1]:
import pandas as pd 
# feature engineering to integrate  column 'id' with index column
df = pd.read_csv("train.csv", index_col='id')
df_test = pd.read_csv("test.csv", index_col='id')
df_sample_submission = pd.read_csv("sample_submission.csv")

## 0. Check the structure of the dataset

#### 0.1 Check missing values

In [ ]:
print(df.isnull().sum())

#### 0.2 Visualize first 5 rows 

In [ ]:
df.head()

#### 0.3 Check how many samples in each class

In [ ]:
import numpy as np
for i in np.unique(df["Response"]):
    print("There are %d samples in train data for category [%s]." % (np.sum(df["Response"] == i), str(i)))

# class imbalance exist

## 1. Feature Preprocessing

### 1.1 Make Categorical 

In [2]:
import numpy as np 
def preprocess(X):
    # creates a copy of X to avoid altering the originial data
    X = X.copy()
    # We use label encoding 
    X['Gender'] = (X['Gender']=='Male').astype(np.uint8)

    # We use boolean mask
    X['Vehicle_Damage'] = (X['Vehicle_Damage'] == 'Yes').astype(np.uint8)

    # We use label encoding
    X["Vehicle_Age"] = X["Vehicle_Age"].astype('category').cat.rename_categories({
            "1-2 Year": 1, "< 1 Year": 0, "> 2 Years": 2}).astype('int8')
    X['Age'] = X['Age'].astype('int8')
    X['Driving_License'] = X['Driving_License'].astype('int8')
    X['Region_Code'] = X['Region_Code'].astype('int8')
    X['Previously_Insured'] = X['Previously_Insured'].astype('int8')
    X['Annual_Premium'] = X['Annual_Premium'].astype('int32')
    X['Policy_Sales_Channel'] = X['Policy_Sales_Channel'].astype('int16')
    X['Vintage'] = X['Vintage'].astype('int16')
    return X
X_preprocess = preprocess(df)

### 1.2 Box Plots (compare the distribution of a continuous feature across different classes)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the boxplots in a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

sns.boxplot(x='Response', y='Age', data=df, ax=axs[0, 0])
axs[0, 0].set_title('Box Plot of Age by Response')
axs[0, 0].set_ylabel("Age")

sns.boxplot(x='Response', y='Annual_Premium', data=df, ax=axs[0, 1])
axs[0, 1].set_title('Box Plot of Annual Premium by Response')
axs[0, 1].set_ylabel("Annual_Premium")

sns.boxplot(x='Response', y='Policy_Sales_Channel', data=df, ax=axs[1, 0])
axs[1, 0].set_title('Box Plot of Policy Sales Channel by Response')
axs[1, 0].set_ylabel("Policy_Sales_Channel")

sns.boxplot(x='Response', y='Vintage', data=df, ax=axs[1, 1])
axs[1, 1].set_title('Box Plot of Vintage by Response')
axs[1, 1].set_ylabel("Vintage")

for ax in axs.flat:
    ax.set_xlabel('Response')

plt.tight_layout()
plt.show()

### 1.3 Bar Plot( Show the count of each categorical data)

In [ ]:
variables = ['Gender', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage']

# Set up the matplotlib figure and axis grid
fig, axes = plt.subplots(3, 2, figsize=(14, 12))
fig.suptitle('Bar Plots of Categorical Variables', fontsize=16)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Iterate over the variables and create a bar plot for each
for ax, var in zip(axes, variables):
    sns.countplot(x=var, hue = "Response", data=X, ax=ax, palette='viridis')
    ax.set_title(f'Bar Plot of {var}')
    ax.set_xlabel(var)
    ax.set_ylabel('Count')

# Adjust layout
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust the rect to make room for the suptitle
plt.show()

## 2. Data Preprocessing

### 2.1 Train, test, valid split

In [3]:
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, train_test_split

X, X_test = train_test_split(X_preprocess , test_size=100000, random_state = 31, stratify = X_preprocess['Response'])
X_train, X_valid = train_test_split(X, test_size=100000, random_state=31, stratify=X['Response'])
# pop('Response'): extract the 'Response' column and remove it from df
y_train = X_train.pop('Response')
y_valid = X_valid.pop('Response')
# X_test = X_valid; y_test = y_valid
y_test = X_test.pop('Response')
# training set is used to train the model(largest)
# Validation set is used to evaluate model performance, and tune the hyperparameters of the model
# test set is used to provide an unbiased estimate of the model's performance on unseen data

### 2.2 Data structure conversion -- DMatrix

In [4]:
import cupy as cp # what is sparse matrix?
import xgboost as xgb # import entire xgboost library

# DMatrix is a data structure optimized for XGBoost, can be use in both device : CPU/GPU
def prepare_DMatrix(training_set, validation_data):
    # convert training data to CuPy arrays
    
    training_set_gpu = cp.array(training_set)
    validation_data_gpu = cp.array(validation_data)
    
    DMatrix = xgb.DMatrix(training_set_gpu, label = validation_data_gpu)

    return DMatrix


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2.3 Downsampling of data

In [5]:
def down_sampling(X, y, i):
    majority_class = X[y == 0].copy() # Extract rows where Response = 0
    minority_class = X[y == 1].copy() # Extract rows where Response = 1
    sample_size = len(minority_class) # Size of the minority class
    # Train_test_split is used to split the majority class into 2 parts
    majority_sample, X_rest, y_sample, y_rest = train_test_split(majority_class, y[y == 0], 
                                                                  train_size=sample_size, random_state=i, 
                                                stratify=majority_class['Age'])
    # train_size/test_size: If int, represents the absolute number of train samples.
    # If float, (0.0,1.0), represent the proportion of the dataset to include in the split
    # If None, automatically set to the complement of the test size

    # majority sample is a subset of majority class, same size as minority class, stratified by age

    # X_minimial combines "majority sample" and "minority class" to create a balanced subset
    X_minimal = pd.concat([majority_sample, minority_class], axis=0)
    # y_minimial combines the target variables of both "majority sample" and "minority class"
    y_minimal = pd.concat([y_sample, y[y == 1]], axis=0)


    return X_minimal, y_minimal, X_rest, y_rest

In [6]:
X_minimal, y_minimal, X_rest, y_rest = down_sampling(X_train, y_train, 63) # random_state is 63

### 2.3.1 Performance of a downsampled dataset

Now we measure how does XGBoost performs in a downsampled, balanced dataset, compare to a random sampling dataset of the same size

In [ ]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_bin': 262143,
    'tree_method': 'hist',
    'device' : 'cuda', 
    'seed' : 42
}

In [ ]:

test_scores, test_scores2 = [], []

dvalid = prepare_DMatrix(X_valid, y_valid)
dtest = prepare_DMatrix(X_test, y_test)

dminimal = prepare_DMatrix(X_minimal, y_minimal)
# Training with early stopping
bst = xgb.train(params, dminimal, num_boost_round=2000, evals=[(dvalid, 'validation')],
                early_stopping_rounds=100, verbose_eval=False)

# Predict probabilities for the test set
y_pred_proba = bst.predict(dtest)

# Evaluate and append ROC AUC score
roc_auc_downsample = roc_auc_score(y_test, y_pred_proba)
test_scores.append(roc_auc_downsample)
print(f"ROC AUC Downsample Score: {roc_auc_downsample}")

# clean up memory
del bst, dminimal
gc.collect()

# random sampling 
X_t2, _, y_t2, _ = train_test_split(X_train, y_train, train_size=2780918, 
                                  random_state=31)

drandom = prepare_DMatrix(X_t2, y_t2)
bst2 = xgb.train(params, drandom, num_boost_round=2000, evals=[(dvalid, 'validation')], # evalation set for determining early stopping
                early_stopping_rounds=100, verbose_eval=False)


y_pred_proba_2 = bst2.predict(dtest)

roc_auc_random = roc_auc_score(y_test, y_pred_proba_2)
test_scores2.append(roc_auc_random)
print(f"ROC AUC Random Score: {roc_auc_random}")

del bst2, X_t2, y_t2, drandom, dtest, dvalid
gc.collect()
# check if adding tree_method='gpu_hist', predictor='gpu_predictor' increase the ROC_AUC_Score
# By default, XGBClassifier uses a threshold of 0.5 for classification.
# We can change the threshold after training to improve metrics such as precision, recall, and F1-score

We have about 8.5 million additional data:

In [ ]:
len(X_rest)

###  2.3.2 Progressively add more samples

In [ ]:
import time
from tqdm import trange

dvalid = prepare_DMatrix(X_valid, y_valid)
dtest = prepare_DMatrix(X_test, y_test)

for n in trange(1, 9):
    start_time = time.time()  # Timing starts
    
    if n * 1000000 < len(X_rest):
        X_t, _, y_t, _ = train_test_split(X_rest, y_rest, train_size=n * 1000000, random_state=31)
        X_t = pd.concat([X_minimal, X_t], axis=0)
        y_t = pd.concat([y_minimal, y_t], axis=0)
        X_t2, _, y_t2, _ = train_test_split(X_train, y_train, train_size=n * 1000000 + 2780918, random_state=31)
    else:
        X_t, y_t = X_train, y_train
        X_t2, y_t2 = X_train, y_train

    print(f"Preparing DMatrix for n={n}")  # Logging preparation
    dMatrix_temporary = prepare_DMatrix(X_t, y_t)
    dMatrix_temporary_2 = prepare_DMatrix(X_t2, y_t2)
    # Train new instances of model to ensure Isolation of experiment
    clf = xgb.train(params, dMatrix_temporary, num_boost_round=2000, evals=[(dvalid, 'validation')],
                    early_stopping_rounds=100, verbose_eval=False)
    clf2 = xgb.train(params, dMatrix_temporary_2, num_boost_round=2000, evals=[(dvalid, 'validation')], 
                     early_stopping_rounds=100, verbose_eval=False)

    test_scores.append(roc_auc_score(y_test, clf.predict(dtest)))
    test_scores2.append(roc_auc_score(y_test, clf2.predict(dtest)))

    del clf, clf2, dMatrix_temporary, dMatrix_temporary_2
    gc.collect()
    
    end_time = time.time()  # Timing ends
    print(f"Iteration {n} completed in {end_time - start_time:.2f} seconds")  # Logging time taken

del dtest, dvalid
gc.collect()

#### 2.4 Save the score in file using pickle

In [ ]:
import pickle

# Save test_scores
with open('test_scores.pkl', 'wb') as f:
    pickle.dump(test_scores, f)

# Save test_scores2
with open('test_scores2.pkl', 'wb') as f:
    pickle.dump(test_scores2,f)

#### 2.4,1 load test scores from pickle

In [ ]:
import pickle

# Load test_scores
with open('test_scores.pkl', 'rb') as f:
    test_scores = pickle.load(f)

# Load test_scores2
with open('test_scores2.pkl', 'rb') as f:
    test_scores2 =pickle.load(f)

### 2.5 Visualization

In [ ]:
import matplotlib.pyplot as plt
n = [0,1,2,3,4,5,6,7,8]
plt.plot(n, test_scores2, marker='o', linestyle='-', color='r', label='random subsample')
plt.plot(n, test_scores, marker='o', linestyle='-', color='b',  label='downsampling')
plt.xlabel('Number of samples (x10^6)')
plt.ylabel('Test Auc Scores')
plt.title('Performance Gain of Adding data rows with Response = 0')
plt.grid(True)
plt.legend()
plt.show()

## 3. XGBoost

### 3.0 XGBoost Base Params

In [ ]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_bin': 262143,
    'tree_method': 'hist',
    'device' : 'cuda', 
    'seed' : 42
}

We set the Max_bin to be unusually high, becuause in an unbalanced dataset, features of the majority class often overshadow those minority class ones.

Instead of grouping all values of a feature into a few broad bins, the model can create many narrow bins, capturing subtle variations in the data.

For example, if a feature slightly differs between the minority and majority classes, a higher max_bin value can help the model recognize these differences more accurately, which might not be possible with fewer bins.

### 3.1 XGBoost model evaluation function 

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix

def evaluate_model_performance_xgboost(model, dvalid, y_valid, threshold=0.5):
    # Predict probabilities for the validation set
    y_valid_pred_proba = model.predict(dvalid)
    y_valid_pred = (y_valid_pred_proba >= threshold).astype(int)
    
    # Calculate the confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_valid, y_valid_pred).ravel()
    
    # Calculate True Positive Rate (TPR) and True Negative Rate (TNR)
    sensitivity = tp / (tp + fn)
    specificity = tn / (fp + tn)
    precision = tp / (tp + fp)
    negative_precision = tn / (tn + fn) #the proportion of true negatives among all predicted negatives

    # Calculate AUC-ROC score
    roc_auc = roc_auc_score(y_valid, y_valid_pred_proba)
    
    # Print the evaluation metrics
    print('Sensitivity (TPR):', sensitivity)
    print('Specificity (TNR):', specificity)
    print('Precision:', precision)
    print('Negative Precision:', negative_precision)
    print('AUC-ROC Score:', roc_auc)

    return sensitivity, specificity, precision, negative_precision


### 3.2 XGBoost, downsampled  balanced dataset, with base params

In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve
import gc


dvalid = prepare_DMatrix(X_valid, y_valid)
# dvalid_selected = prepare_DMatrix(X_valid_selected, y_valid)
dminimal = prepare_DMatrix(X_minimal, y_minimal)
# dminimal_selected = prepare_DMatrix(X_minimal_5_selected, y_minimal)

# Training with early stopping
bst = xgb.train(params, dminimal, num_boost_round=2000, evals=[(dvalid, 'validation')],
                early_stopping_rounds=100, verbose_eval=False)

evaluate_model_performance_xgboost(bst, dvalid, y_valid, threshold = 0.5)

# Clean up memory
del bst,dvalid,dminimal,
gc.collect()

### 3.3 XGBoost, downsampled , balanced + 3000000 (label = 0) dataset, with base params

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import gc

# Split and combine the data
X_3mil, _, y_3mil, _ = train_test_split(X_rest, y_rest, train_size=3 * 1000000, random_state=31, stratify=y_rest)

# Combine minimal dataset with additional samples
X_concat_3mil = pd.concat([X_minimal, X_3mil], axis=0)
y_concat_3mil = pd.concat([y_minimal, y_3mil], axis=0)


dvalid = prepare_DMatrix(X_valid, y_valid)
# dvalid_selected = prepare_DMatrix(X_valid_selected, y_valid)
d_concat_3mil = prepare_DMatrix(X_concat_3mil, y_concat_3mil)

# Training with early stopping
bst2 = xgb.train(params, d_concat_3mil, num_boost_round=2000, evals=[(dvalid, 'validation')],
                early_stopping_rounds=100, verbose_eval=False)

evaluate_model_performance_xgboost(bst2, dvalid, y_valid, threshold = 0.5)
del dvalid,d_concat_3mil
gc.collect()


## 4. Feature Analysis

### 4.1 Feature Importance (gain)

In [ ]:
bst2.feature_names = feature_names
xgb.plot_importance(bst2, importance_type='gain')
plt.title('Feature Importance (Gain)')
plt.show()

**gain**: gain in AUC_ROC score

In Feature of importance (gain), the top 2 features are **"Previously_Insured"** and **"Vehicle_Damage"**. These features have the highest importance scores, indicating they contribute the most to the model's predictive power in terms of gain.



### 4.2 Feature Importance (weight)

In [ ]:
xgb.plot_importance(bst2, importance_type='weight')
plt.title('Feature Importance (Weight)')
plt.show()

**weight**: most commonly used to separate nodes, not directly related to increase in AUC_ROC score

The top features by weight are **"Annual_Premium"** and **"Vintage"**.


### 4.3 Shap

### 5. Feature Engineering -- Removal of Features

### 5.1 Baseline AUC-ROC score

In [ ]:
# Assume df is your DataFrame and it contains the target and features
# Replace 'target' with your actual target column name
feature_names = [
    'Gender','Age','Driving_License','Region_Code','Previously_Insured','Vehicle_Age','Vehicle_Damage','Annual_Premium','Policy_Sales_Channel','Vintage']
# Split and combine the data
X_3mil, _, y_3mil, _ = train_test_split(X_rest, y_rest, train_size=3 * 1000000, random_state=31, stratify=y_rest)

# Combine minimal dataset with additional samples
X_concat_3mil = pd.concat([X_minimal, X_3mil], axis=0)
y_concat_3mil = pd.concat([y_minimal, y_3mil], axis=0)


dvalid = prepare_DMatrix(X_valid, y_valid)
# dvalid_selected = prepare_DMatrix(X_valid_selected, y_valid)
d_concat_3mil = prepare_DMatrix(X_concat_3mil, y_concat_3mil)

# Dictionary to store the ROC AUC scores for each dropped predictor
auc_scores = {}

# Evaluate baseline model with all predictors
bst_all = xgb.train(params, d_concat_3mil, num_boost_round=2000, evals=[(dvalid, 'validation')],
                    early_stopping_rounds=100, verbose_eval=False)

preds_all = bst_all.predict(dvalid)
auc_all = roc_auc_score(y_valid, preds_all)
auc_scores["without_dropping"] = auc_all
print(f'Baseline AUC with all features: {auc_all:.4f}')

del dvalid,d_concat_3mil, bst_all
gc.collect()

### 5.2 AUC score with 1 predictor removed per iteration

In [ ]:

# Iterate over each predictor and evaluate performance without it
for predictor in feature_names:
    # Create training data without the current predictor
    reduced_features = [f for f in feature_names if f != predictor]
    X_concat_3mil_reduced = X_concat_3mil[reduced_features]
    X_valid_reduced = X_valid[reduced_features]

    # Prepare DMatrices
    d_concat_3mil_reduced = prepare_DMatrix(X_concat_3mil_reduced, y_concat_3mil)
    dvalid_reduced = prepare_DMatrix(X_valid_reduced, y_valid)

    # Train the model
    bst_reduced = xgb.train(params, d_concat_3mil_reduced, num_boost_round=2000, evals=[(dvalid_reduced, 'validation')],
                            early_stopping_rounds=100, verbose_eval=False)

    # Predict and evaluate
    preds_reduced = bst_reduced.predict(dvalid_reduced)
    auc_reduced = roc_auc_score(y_valid, preds_reduced)

    auc_scores[predictor] = auc_reduced
    print(f'AUC without {predictor}: {auc_reduced:.4f}')

    del dvalid_reduced ,d_concat_3mil_reduced, bst_reduced
    gc.collect()
# Find the predictor with the highest AUC score
best_predictor = max(auc_scores, key=auc_scores.get)
highest_auc_score = auc_scores[best_predictor]

# Print the best predictor and its AUC score
print(f"Highest AUC score with predictor removed: {best_predictor}: {highest_auc_score:.4f}")



AUC without **"Annual_Premium"** and **"Vintage"** decreases the most, suggesting these are the most important predictors for the target variables.

AUC without **"Driving_License"** increases the most, possibly indicating that this feature might be less informative or noisy.


#### 5.3 AUC score with "Driving_License"+ 1 predictor removed 

In [ ]:
# baseline model, only drops the "Driving License"
feature_names_without_Driving_License = [
    'Gender','Age','Region_Code','Previously_Insured','Vehicle_Age','Vehicle_Damage','Annual_Premium','Policy_Sales_Channel','Vintage']
# Define the model parameters
params = {'objective': 'binary:logistic', 'eval_metric': 'auc','max_bin': 262143,'tree_method': 'hist','device' : 'cuda','seed' : 42}
# Dictionary to store the ROC AUC scores for each dropped predictor
auc_scores_remove_2_predictors = {}

# Split and combine the data
X_3mil, _, y_3mil, _ = train_test_split(X_rest, y_rest, train_size=3 * 1000000, random_state=31, stratify=y_rest)

# Combine minimal dataset with additional samples
X_concat_3mil = pd.concat([X_minimal, X_3mil], axis=0)
y_concat_3mil = pd.concat([y_minimal, y_3mil], axis=0)

X_concat_3mil_no_Driving_License = X_concat_3mil.drop("Driving_License", axis=1)
X_valid_no_Driving_License = X_valid.drop("Driving_License", axis=1)

dvalid_no_Driving_License = prepare_DMatrix(X_valid_no_Driving_License , y_valid)
# dvalid_selected = prepare_DMatrix(X_valid_selected, y_valid)
d_concat_3mil_no_Driving_License = prepare_DMatrix(X_concat_3mil_no_Driving_License, y_concat_3mil)

bst_all = xgb.train(params, d_concat_3mil_no_Driving_License, num_boost_round=2000, evals=[(dvalid_no_Driving_License , 'validation')],
                    early_stopping_rounds=100, verbose_eval=False)

preds_all = bst_all.predict(dvalid_no_Driving_License )
auc_all = roc_auc_score(y_valid, preds_all)
auc_scores_remove_2_predictors["baseline_drop_Driving_License"] = auc_all
print(f'Baseline AUC , only drops the "Driving License": {auc_all:.4f}')

del dvalid_no_Driving_License ,d_concat_3mil_no_Driving_License, bst_all
gc.collect()


# Iterate over each predictor and evaluate performance without it
for predictor in feature_names_without_Driving_License:
    # Create training data without the current predictor
    reduced_features = [f for f in feature_names_without_Driving_License if f != predictor]
    X_concat_3mil_reduced = X_concat_3mil[reduced_features]
    X_valid_reduced = X_valid[reduced_features]

    # Prepare DMatrices
    d_concat_3mil_reduced = prepare_DMatrix(X_concat_3mil_reduced, y_concat_3mil)
    dvalid_reduced = prepare_DMatrix(X_valid_reduced, y_valid)

    # Train the model
    bst_reduced = xgb.train(params, d_concat_3mil_reduced, num_boost_round=2000, evals=[(dvalid_reduced, 'validation')],
                            early_stopping_rounds=100, verbose_eval=False)

    # Predict and evaluate
    preds_reduced = bst_reduced.predict(dvalid_reduced)
    auc_reduced = roc_auc_score(y_valid, preds_reduced)

    auc_scores_remove_2_predictors[predictor] = auc_reduced
    print(f'AUC without "Driving_License" + {predictor}: {auc_reduced:.4f}')

    del dvalid_reduced ,d_concat_3mil_reduced, bst_reduced
    gc.collect()
# Find the predictor with the highest AUC score
best_predictor = max(auc_scores_remove_2_predictors, key=auc_scores_remove_2_predictors.get)
highest_auc_score = auc_scores_remove_2_predictors[best_predictor]

# Print the best predictor and its AUC score
print(f"Highest AUC score with 'Driving_License' + predictor removed: {best_predictor}: {highest_auc_score:.4f}")

AUC without **"Driving_License"** remains the highest among all combinations of removal with "Driving_License" included. This indicates that we won't need to further remove features.


### 6. Feature Engineering -- Combining Features to increase predictive power

6.1 We start with combining the most predictive feature — **"Previously_Insured"**, with one of the other predictors in each iteration.


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import gc

# Example data preparation
# Assuming X_minimal, X_rest, y_minimal, y_rest, X_valid, and y_valid are already defined
# Dictionary to store AUC scores
auc_scores = {}

# Define the model parameters

# Combine datasets
X_3mil, _, y_3mil, _ = train_test_split(X_rest, y_rest, train_size=3 * 1000000, random_state=31, stratify=y_rest)
X_concat_3mil = pd.concat([X_minimal, X_3mil], axis=0)
y_concat_3mil = pd.concat([y_minimal, y_3mil], axis=0)

# List of feature names excluding 'Driving_License'
feature_names_without_Driving_License = [
    'Gender', 'Age', 'Region_Code', 'Previously_Insured', 
    'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 
    'Policy_Sales_Channel', 'Vintage'
]

# Ensure 'Driving_License' is not in the data
X_concat_3mil_no_Driving_License = X_concat_3mil.drop("Driving_License", axis=1)
X_valid_no_Driving_License = X_valid.drop("Driving_License", axis=1)


feature_names_no_Prev = [
    'Gender', 'Age', 'Region_Code', 
    'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 
    'Policy_Sales_Channel', 'Vintage'
]

# Iterate over each feature and evaluate performance with Previously_Insured
for feature in feature_names_no_Prev:
    # Create a new combined feature
    combined_feature_name = f'Previously_Insured_{feature}'

    # avoid changing the originial datasets
    df1 = X_concat_3mil_no_Driving_License.copy()
    df_valid1 = X_valid_no_Driving_License.copy()

# Create the new combined feature
    df1[combined_feature_name] = pd.factorize(df1['Previously_Insured'].astype(str) + '_' + df1[feature].astype(str))[0] # 8 features + 1 combined feature
    df_valid1[combined_feature_name] = pd.factorize(df_valid1['Previously_Insured'].astype(str) + '_' + df_valid1[feature].astype(str))[0] #  pd.factorize(array) returns a tuple of (codes, uniques). Since we are just interested in codes, we use [0]

    # Prepare DMatrices
    d_combined_3mil = prepare_DMatrix(df1, y_concat_3mil)
    dvalid_combined =  prepare_DMatrix(df_valid1, y_valid)

    # Train the model
    bst_combined = xgb.train(params, d_combined_3mil, num_boost_round=2000,
                             evals=[(dvalid_combined, 'validation')],
                             early_stopping_rounds=100, verbose_eval=False)

    # Predict and evaluate
    preds_combined = bst_combined.predict(dvalid_combined)
    auc_combined = roc_auc_score(y_valid, preds_combined)

    auc_scores[feature] = auc_combined
    print(f'AUC score combining Previously_Insured and {feature}: {auc_combined:.4f}')

    # Clean up to free memory
    del dvalid_combined, d_combined_3mil, bst_combined
    gc.collect()

# Find the feature with the highest AUC score when combined with Previously_Insured
best_feature = max(auc_scores, key=auc_scores.get)
highest_auc_score = auc_scores[best_feature]

# Print the best feature and its AUC score
print(f"Highest AUC score combining 'Previously_Insured' and {best_feature}: {highest_auc_score:.4f}")


Highest AUC score is from the baseline model, without combining features 

6.2 The second approach is based on the correlation matrix. We combine two less correlated features, which may provide complementary information when combined, hence increasing the AUC-ROC score.


## 7. Fine tuning of XGBoost model

#### 7.1 learning rate (default eta = 0.3 is the best )

### 7.2  scale_pos_weight [default=1] 

The `scale_pos_weight` parameter in XGBoost is specifically applied to the positive class. In the context of an imbalanced dataset (positive: 1, negative: 10), adjusting this parameter can improve the model's sensitivity to the positive class.

During training, 60% of the data is used from X_rest. The typical value for `scale_pos_weight` is approximately 4.678, calculated as the ratio of negative to positive instances.


In [ ]:
import optuna
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import gc

# Define the objective function for Optuna
def objective(trial):
    # Suggest a value for scale_pos_weight
    scale_pos_weight = trial.suggest_float('scale_pos_weight', 1, 1.20)

    # Update the parameters with the suggested scale_pos_weight
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'max_bin': 262143,
        'tree_method': 'hist',
        'device': 'cuda',
        'min_child_weight' : 2.5,
        'scale_pos_weight': scale_pos_weight
    }
    X_rest_0_6, _, y_rest_0_6, _ = train_test_split(X_rest, y_rest, train_size=0.6, random_state=31, stratify=y_rest)

    X_imbalanced = pd.concat([X_minimal, X_rest_0_6], axis=0)
    y_imbalanced = pd.concat([y_minimal, y_rest_0_6], axis=0)
    
    X_imbalanced = X_imbalanced.drop("Driving_License", axis=1)
    X_valid_imbalanced = X_valid.drop("Driving_License", axis=1)

    d_imbalanced = prepare_DMatrix(X_imbalanced, y_imbalanced)
    d_val = prepare_DMatrix(X_valid_imbalanced, y_valid)

    # Train the model
    model = xgb.train(params, d_imbalanced, num_boost_round=2000, evals=[(d_val, 'validation')],
                      early_stopping_rounds=100, verbose_eval=False)

    # Predict and evaluate the model
    y_pred = model.predict(d_val)
    score = roc_auc_score(y_valid, y_pred)

    del d_imbalanced,d_val,model
    gc.collect()
    return score

# Set up Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Get the best trial and its parameters
best_trial = study.best_trial
best_scale_pos_weight = best_trial.params['scale_pos_weight']
best_score = best_trial.value

print(f"Best scale_pos_weight: {best_scale_pos_weight} with AUC-ROC: {best_score:.5f}")

### 7.3 Max_depth (default=6 is the best)  

Maximum depth of tree, where increasing it would increase the risk of overfitting

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gc

def find_best_max_depth(base_params, max_depth_values):
    best_max_depth = None
    best_score = -np.inf
    best_oof_preds = None
    max_depth_auc_roc_scores = {}

    for max_depth in max_depth_values:
        print(f"Testing max_depth={max_depth}")
        params = base_params.copy()
        params['max_depth'] = max_depth

        # Down-sample the majority class
        X_rest_0_6, _, y_rest_0_6, _ = train_test_split(X_rest, y_rest, train_size=0.6, random_state=31, stratify=y_rest)

        X_imbalanced = pd.concat([X_minimal, X_rest_0_6], axis=0)
        y_imbalanced = pd.concat([y_minimal, y_rest_0_6], axis=0)

        X_imbalanced = X_imbalanced.drop("Driving_License", axis=1)
        X_valid_imbalanced = X_valid.drop("Driving_License", axis=1)

        d_imbalanced = prepare_DMatrix(X_imbalanced, y_imbalanced)
        d_val = prepare_DMatrix(X_valid_imbalanced, y_valid)

        # Train the model
        model = xgb.train(params, d_imbalanced, num_boost_round=2000, evals=[(d_val, 'validation')],
                          early_stopping_rounds=100, verbose_eval=False)

        # Predict and evaluate the model
        y_pred = model.predict(d_val)
        score = roc_auc_score(y_valid, y_pred)
        max_depth_auc_roc_scores[max_depth] = score
        print(f"ROC-AUC for max_depth={max_depth}: {score:.5f}")

        if score > best_score:
            best_score = score
            best_max_depth = max_depth
            best_oof_preds = y_pred

        del d_imbalanced, d_val, model
        gc.collect()

    print(f"Best max_depth: {best_max_depth} with ROC-AUC: {best_score:.5f}")
    return best_max_depth, best_oof_preds, max_depth_auc_roc_scores

# Plotting function
def plot_max_depth_vs_roc_auc(max_depth_auc_roc_scores):
    max_depth_values = list(max_depth_auc_roc_scores.keys())
    roc_auc_scores = list(max_depth_auc_roc_scores.values())

    plt.figure(figsize=(10, 6))
    plt.plot(max_depth_values, roc_auc_scores, marker='o', linestyle='-', color='b', label='ROC-AUC')
    optimal_index = np.argmax(roc_auc_scores)
    optimal_max_depth = max_depth_values[optimal_index]
    optimal_auc = roc_auc_scores[optimal_index]
    plt.scatter(optimal_max_depth, optimal_auc, color='r', label=f'Optimal max_depth: {optimal_max_depth}', zorder=5)
    plt.title('max_depth vs. ROC-AUC')
    plt.xlabel('max_depth')
    plt.ylabel('ROC-AUC')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_bin': 262143,
    'tree_method': 'hist',
    'device': 'cuda'
}

# Define the range of max_depth values to test
max_depth_values = range(3, 11)

# Call the function
best_max_depth, oof_preds, max_depth_auc_roc_scores = find_best_max_depth(params, max_depth_values)
plot_max_depth_vs_roc_auc(max_depth_auc_roc_scores)


### 7.4 Scale min_child_weight [default = 1]

It specifies the minimum sum of instance weight (hessian) needed in a child node.

It is a form of regularization to prevent overfitting

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import gc


def find_best_min_child_weight(base_params, min_child_weight_values):
    best_min_child_weight = None
    best_score = -np.inf
    best_oof_preds = None
    best_test_preds = None
    min_child_weight_auc_roc_scores = {}

    for min_child_weight in min_child_weight_values:
        print(f"Testing min_child_weight={min_child_weight}")
        params = base_params.copy()
        params['min_child_weight'] = min_child_weight

        # Down-sample the majority class
        X_rest_0_6, _, y_rest_0_6, _ = train_test_split(X_rest, y_rest, train_size=0.6, random_state=31, stratify=y_rest)

        X_imbalanced = pd.concat([X_minimal, X_rest_0_6], axis=0)
        y_imbalanced = pd.concat([y_minimal, y_rest_0_6], axis=0)

        X_imbalanced = X_imbalanced.drop("Driving_License", axis=1)
        X_valid_imbalanced = X_valid.drop("Driving_License", axis=1)

        d_imbalanced = prepare_DMatrix(X_imbalanced, y_imbalanced)
        d_val = prepare_DMatrix(X_valid_imbalanced, y_valid)

        model = xgb.train(params, d_imbalanced, num_boost_round=2000, evals=[(d_val, 'validation')],
                          early_stopping_rounds=100, verbose_eval=False)
        y_pred = model.predict(d_val)
        score = roc_auc_score(y_valid, y_pred)
        min_child_weight_auc_roc_scores[min_child_weight] = score
        print(f"ROC-AUC for min_child_weight={min_child_weight}: {score:.5f}")

        if score > best_score:
            best_score = score
            best_min_child_weight = min_child_weight
            best_oof_preds = y_pred

        del d_imbalanced, d_val
        gc.collect()

    print(f"Best min_child_weight: {best_min_child_weight} with ROC-AUC: {best_score:.5f}")

    # Base parameters for XGBoost model
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_bin': 262143,
    'tree_method': 'hist',
    'device' : 'cuda'
}

# Define the range of min_child_weight values to test
min_child_weight_values = [0.3,0.5, 1,1.5,2, 2.5]
# 3.5 ,4, 4.5,5
# Call the function to find the best min_child_weight
find_best_min_child_weight(params, min_child_weight_values)

## 8. Cross_validation + bagging 

--> XGBoost model only

--> remove feature "Driving_License"

--> with newly tuned hyperparameters

In [7]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_bin': 262143,
    'tree_method': 'hist',
    'device' : 'cuda',
    # 'eta' : 0.3 (default)
    # 'max_depth' : 6 (default)
    'scale_pos_weight' : 1.1705442976059786,
    # 'min_child_weight' : 1
}

In [8]:
import datetime
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import gc
# for binary classification, use predict() instead of predict_proba

def cross_validate(X ,y, test_data, n_splits = 10, n_bags = 3 ):

    start_time = datetime.datetime.now()
    scores = []
    oof_preds = np.zeros(len(y))
    test_preds = np.zeros(len(test_data))
    
    kfold = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state=63)

    d_test_csv = prepare_DMatrix(test_data, np.zeros(test_data.shape[0])) # generate a DMatrix of test_data without labels = 0, does not affect prediction

    for fold, (train_index, valid_index) in enumerate(kfold.split(X,y.astype(str))):
        for i in range(0,n_bags):                                        
            X_train = X.iloc[train_index]
            y_train = y.iloc[train_index]
            X_val = X.iloc[valid_index]
            y_val = y.iloc[valid_index]
            # bagging is aggregating the results predicted by different sub-trees (in XGBoost)
            # We train the model using different, balanced, sub-datasets, with a random state = '10*fold + i'
            X_train, y_train, X_rest, y_rest = down_sampling(X_train, y_train, 10*fold + i)
            # double random to ensure the model capture more diverse patterns
            X_rest_0_6, _, y_rest_0_6, _ = train_test_split(X_rest, y_rest, train_size=0.6, random_state=10*fold + i, stratify=y_rest)  #Increase train size to 0.9
            # add 60% of label = 0 data into training 
            X_imbalanced = pd.concat([X_train, X_rest_0_6], axis=0)
            y_imbalanced = pd.concat([y_train, y_rest_0_6], axis=0)
            

            d_imbalanced = prepare_DMatrix(X_imbalanced, y_imbalanced)
            d_val = prepare_DMatrix(X_val, y_val)

            m = xgb.train(params, d_imbalanced, num_boost_round=2000, evals=[(d_val, 'validation')],
                early_stopping_rounds=100, verbose_eval=False)
            y_pred = m.predict(d_val) # returns a 1D array of probability for Response == 1
            # Measures the ability of the model to discriminate between positive and negative classes based on predicted probabilities
            # Useful in imbalanced dataset
            score = roc_auc_score(y_val,  y_pred)
            print(f"# Fold {fold}, bag {i}: ROC-AUC-Score={score:.5f}")
            scores.append(score)
            # in stratified k-fold, every sample will ultimately be used as the validation set
            # And for every index in the validation set, our overall prediction is based on averaging the predictions of 3 subtrees (n_bags = 3)
            # So we divide the predictions of each subtree by 3(n_bags = 3), and sum the predictions
            oof_preds[valid_index] += y_pred/n_bags
            # calculate the test predictions base on the subtree created in current fold
            # overall test prediction = average of 5 folds(n_splits = 5), each fold has 3 subtrees(n_bags = 3)
            # So we divide the predictions of each subtree by 3(n_bags = 3),and further duvude it by the number of folds (n_splits = 5), and finally sum the predictions
            
            test_preds = test_preds + m.predict(d_test_csv)/kfold.get_n_splits()/n_bags
            
            del d_imbalanced, d_val
            gc.collect()

    del d_test_csv
    gc.collect()
    
    elapsed_time = datetime.datetime.now() - start_time
    print(f"#ROC-AUC mean: {np.array(scores).mean():.7f} (+- {np.array(scores).std():.7f})"
              f"#   elapsed time:   {int(np.round(elapsed_time.total_seconds() / 60))} min")

    return oof_preds, test_preds
# but because of the huge class imbalance, in each fold the majority class is way more than the minority class, 
# so even though we use bagging with different random states, to choose different samples from the majority class, 
# there will still be some unused majority class sample

In [9]:
X = preprocess(df)
y = X.pop("Response")
X_drop_driving_license = X.drop("Driving_License", axis=1)
# print(X_drop_driving_license.head())

test_data = preprocess(df_test)
test_data_drop_driving_license = test_data.drop("Driving_License", axis=1)
# print(test_data_drop_driving_license.head())

xgb_oof, xgb_test = cross_validate(X_drop_driving_license, y,test_data_drop_driving_license, n_splits = 10, n_bags = 3 )
print(f"Total ROC-AUC: {roc_auc_score(y, xgb_oof)}")

# Fold 0, bag 0: ROC-AUC-Score=0.89016
# Fold 0, bag 1: ROC-AUC-Score=0.89010
# Fold 0, bag 2: ROC-AUC-Score=0.89022
# Fold 1, bag 0: ROC-AUC-Score=0.89050
# Fold 1, bag 1: ROC-AUC-Score=0.89057
# Fold 1, bag 2: ROC-AUC-Score=0.89054
# Fold 2, bag 0: ROC-AUC-Score=0.89070
# Fold 2, bag 1: ROC-AUC-Score=0.89082
# Fold 2, bag 2: ROC-AUC-Score=0.89077
# Fold 3, bag 0: ROC-AUC-Score=0.89069
# Fold 3, bag 1: ROC-AUC-Score=0.89066
# Fold 3, bag 2: ROC-AUC-Score=0.89085
# Fold 4, bag 0: ROC-AUC-Score=0.89135
# Fold 4, bag 1: ROC-AUC-Score=0.89133
# Fold 4, bag 2: ROC-AUC-Score=0.89125
# Fold 5, bag 0: ROC-AUC-Score=0.89134
# Fold 5, bag 1: ROC-AUC-Score=0.89132
# Fold 5, bag 2: ROC-AUC-Score=0.89138
# Fold 6, bag 0: ROC-AUC-Score=0.89036
# Fold 6, bag 1: ROC-AUC-Score=0.89055
# Fold 6, bag 2: ROC-AUC-Score=0.89046
# Fold 7, bag 0: ROC-AUC-Score=0.89056
# Fold 7, bag 1: ROC-AUC-Score=0.89065
# Fold 7, bag 2: ROC-AUC-Score=0.89069
# Fold 8, bag 0: ROC-AUC-Score=0.89072
# Fold 8, bag 1: ROC-AUC-

#### Generate submission file

In [10]:
import pandas as pd

# Assuming 'test' is your test DataFrame and 'xgb_test' contains the predicted probabilities
# Ensure 'test' contains the 'id' column with the corresponding IDs
df_test_with_id = pd.read_csv("test.csv")
# Create the submission DataFrame
submission = pd.DataFrame({
    'id': df_test_with_id ['id'],  # Replace 'id' with the appropriate identifier column name
    'Response': xgb_test  # The predicted probabilities
})

# Save the submission DataFrame to a CSV file
submission.to_csv('submission_03.csv', index=False)

# Display the first few rows of the submission DataFrame (optional)
print(submission.head())

         id  Response
0  11504798  0.006490
1  11504799  0.753227
2  11504800  0.351327
3  11504801  0.000147
4  11504802  0.280768


: 

#### Insight: When we use more than 1 downsampling/splitting steps in bagging, we should use different random_state for ***EVERY*** steps